In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, Dropout, Activation
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LambdaCallback
from keras.regularizers import l2
from sklearn.metrics import roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import time
import tqdm
import random
from bayes_opt import BayesianOptimization

Using TensorFlow backend.


In [2]:
X = np.load('generated_data.npy')
y = np.round(X[:,-1])
y[y<0]=0
y[y>1]=1
X=X[:,0:-1]

indices = np.random.permutation(X.shape[0])
train_len = int(round(.6*X.shape[0]))
val_len = int(round(.2*X.shape[0]))
test_len = val_len+1
train_idx = indices[:train_len]
val_idx = indices[train_len:train_len+val_len]
test_idx = indices[train_len+val_len:]
X_train, X_val, X_test = X[train_idx,:], X[val_idx,:], X[test_idx,:]
y_train, y_val, y_test = y[train_idx], y[val_idx], y[test_idx]

#np.save('train.npy', np.hstack((X_train,np.reshape(y_train,(-1,1)))))
#np.save('val.npy', np.hstack((X_val,np.reshape(y_val,(-1,1)))))
#np.save('test.npy',np.hstack((X_test,np.reshape(y_test,(-1,1))))) 

In [3]:
def buildDeep(lr=1e-2, dropoutA=.5, dropoutB=.5, dropoutC=.5, weightReg=.01):
    inputLayer=Input(shape=(8,))
    O = Dense(1024, W_regularizer=l2(weightReg), b_regularizer=l2(weightReg))(inputLayer)
    O = Activation('relu')(O)
    O = Dropout(dropoutA)(O)
    O = Dense(512, W_regularizer=l2(weightReg), b_regularizer=l2(weightReg))(O)
    O = Activation('relu')(O)
    O = Dropout(dropoutB)(O)
    O = Dense(256, W_regularizer=l2(weightReg), b_regularizer=l2(weightReg))(O)
    O = Activation('relu')(O)
    O = Dropout(dropoutC)(O)
    pred = Dense(1, activation='sigmoid')(O)
    model = Model(input=inputLayer, output=pred)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr))
    return model

def trainAndPredict(model, nb_epoch=100, batch_size=64):
    #plotterT = LambdaCallback(on_epoch_end=lambda epoch, logs: plt.plot(np.arange(epoch), logs['loss']))
    #plotterV = LambdaCallback(on_epoch_end=lambda epoch, logs: plt.plot(np.arange(epoch), logs['val_loss']))
    stopper = EarlyStopping(monitor='loss',patience = 3, min_delta=.0001, verbose = 1)
    model.fit(X_train,y_train,validation_data=(X_val,y_val),
              nb_epoch=int(round(nb_epoch)), batch_size=int(round(batch_size)),callbacks=[stopper], verbose=2)
    return model.predict(X_val)

def evalAUC(y_pred,y_val=y_val):
    return roc_auc_score(y_val,y_pred)

def measurableDeep(lr=.01, dropoutA=.5, dropoutB=.5, dropoutC=.5, weightReg=.01, nb_epoch=100, batch_size=64):
    model = buildDeep(lr,dropoutA,dropoutB,dropoutC, weightReg)
    y_pred = trainAndPredict(model, nb_epoch,batch_size)
    return evalAUC(y_pred)

In [14]:
#params = {'lr':(1e-4,.005), 'weightReg':(0,.25)}
#BO = BayesianOptimization(measurableDeep, params)
#BO.maximize(init_points=5,n_iter=50, acq='ei')


Initialization
-------------------------------------------------------
 Step |   Time |      Value |        lr |   weightReg | 
Train on 1208367 samples, validate on 402789 samples
Epoch 1/100
271s - loss: 0.2520 - val_loss: 0.0847
Epoch 2/100
271s - loss: 0.0835 - val_loss: 0.0753
Epoch 3/100
278s - loss: 0.0813 - val_loss: 0.0817
Epoch 4/100
283s - loss: 0.0809 - val_loss: 0.0696
Epoch 5/100
280s - loss: 0.0800 - val_loss: 0.1000
Epoch 6/100
272s - loss: 0.0797 - val_loss: 0.0628
Epoch 7/100
281s - loss: 0.0788 - val_loss: 0.0671
Epoch 8/100
281s - loss: 0.0778 - val_loss: 0.0609
Epoch 9/100
276s - loss: 0.0763 - val_loss: 0.0721
Epoch 10/100
274s - loss: 0.0764 - val_loss: 0.0659
Epoch 11/100
273s - loss: 0.0763 - val_loss: 0.0546
Epoch 12/100
275s - loss: 0.0750 - val_loss: 0.0773
Epoch 13/100
283s - loss: 0.0752 - val_loss: 0.0740
Epoch 14/100
283s - loss: 0.0752 - val_loss: 0.0638
Epoch 15/100
280s - loss: 0.0738 - val_loss: 0.0612
Epoch 16/100
277s - loss: 0.0737 - val_loss: 0.0

/Users/jasonmancuso/anaconda/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -2.53353537e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


   20 | 414m07s |    0.99325 |    0.0001 |      0.0240 | 
Train on 1208367 samples, validate on 402789 samples
Epoch 1/100
477s - loss: 0.2241 - val_loss: 0.0761
Epoch 2/100
603s - loss: 0.1758 - val_loss: 0.0732
Epoch 3/100
603s - loss: 0.1612 - val_loss: 0.0737
Epoch 4/100
589s - loss: 0.1377 - val_loss: 0.0753
Epoch 5/100
600s - loss: 0.1905 - val_loss: 0.0732
Epoch 6/100
603s - loss: 0.1400 - val_loss: 0.0752
Epoch 7/100
601s - loss: 0.1651 - val_loss: 0.0732
Epoch 8/100
601s - loss: 0.1562 - val_loss: 0.0730
Epoch 00007: early stopping
   21 | 78m55s |    0.50000 |    0.0050 |      0.2500 | 
Train on 1208367 samples, validate on 402789 samples
Epoch 1/100
507s - loss: 3.2740 - val_loss: 0.0713
Epoch 2/100
598s - loss: 0.0707 - val_loss: 0.0663
Epoch 3/100
599s - loss: 0.0633 - val_loss: 0.0572
Epoch 4/100
600s - loss: 0.0583 - val_loss: 0.0515
Epoch 5/100
600s - loss: 0.0538 - val_loss: 0.0486
Epoch 6/100
600s - loss: 0.0480 - val_loss: 0.0451
Epoch 7/100
600s - loss: 0.0463 - val

KeyboardInterrupt: 

In [16]:
BO.res['all']

{'params': [{'lr': 0.0001, 'weightReg': 0.16338150567560639},
  {'lr': 0.0050000000000000001, 'weightReg': 0.13203963541895281},
  {'lr': 0.0001, 'weightReg': 0.19797349112494181},
  {'lr': 0.0001, 'weightReg': 0.10385091063317732},
  {'lr': 0.0001, 'weightReg': 0.049436776248922863},
  {'lr': 0.0001, 'weightReg': 0.060095178273605603},
  {'lr': 0.0050000000000000001, 'weightReg': 0.054310381344428484},
  {'lr': 0.0001, 'weightReg': 0.036971179319672087},
  {'lr': 0.0001, 'weightReg': 0.042329115004096432},
  {'lr': 0.0001, 'weightReg': 0.068340835985962939},
  {'lr': 0.0001, 'weightReg': 0.030622023438383127},
  {'lr': 0.0001, 'weightReg': 0.032765241836584802},
  {'lr': 0.0001, 'weightReg': 0.0},
  {'lr': 0.0001, 'weightReg': 0.026254263965213388},
  {'lr': 0.0001, 'weightReg': 0.02397227908686737},
  {'lr': 0.0050000000000000001, 'weightReg': 0.25},
  {'lr': 0.0001, 'weightReg': 0.22133681371660766}],
 'values': [0.80894850639539695,
  0.5,
  0.77760309491634527,
  0.812406798259257

In [36]:
best_params = BO.res['max']['max_params']
print(best_params)

{'lr': 0.0001, 'weightReg': 0.026254263965213388}


In [4]:
#best_model = buildDeep(lr=.0001,weightReg=0.026254263965213388)
#stopper = EarlyStopping(monitor='loss',patience = 3, min_delta=.0001, verbose = 1)
#best_model.fit(X_train,y_train,validation_data=(X_val,y_val),
#              nb_epoch=100, batch_size=64,callbacks=[stopper], verbose=2)
#best_model.save('best_deep_model.hdf5')

Train on 1208367 samples, validate on 402789 samples
Epoch 1/100
258s - loss: 0.7612 - val_loss: 0.0463
Epoch 2/100
259s - loss: 0.0461 - val_loss: 0.0377
Epoch 3/100
258s - loss: 0.0380 - val_loss: 0.0346
Epoch 4/100
259s - loss: 0.0347 - val_loss: 0.0313
Epoch 5/100
259s - loss: 0.0325 - val_loss: 0.0294
Epoch 6/100
260s - loss: 0.0310 - val_loss: 0.0283
Epoch 7/100
259s - loss: 0.0299 - val_loss: 0.0267
Epoch 8/100
259s - loss: 0.0289 - val_loss: 0.0258
Epoch 9/100
260s - loss: 0.0282 - val_loss: 0.0263
Epoch 10/100
259s - loss: 0.0274 - val_loss: 0.0241
Epoch 11/100
259s - loss: 0.0248 - val_loss: 0.0225
Epoch 12/100
259s - loss: 0.0241 - val_loss: 0.0217
Epoch 13/100
259s - loss: 0.0235 - val_loss: 0.0238
Epoch 14/100
259s - loss: 0.0233 - val_loss: 0.0200
Epoch 15/100
259s - loss: 0.0230 - val_loss: 0.0213
Epoch 16/100
259s - loss: 0.0226 - val_loss: 0.0200
Epoch 17/100
259s - loss: 0.0224 - val_loss: 0.0217
Epoch 18/100
259s - loss: 0.0223 - val_loss: 0.0216
Epoch 19/100
259s - 

In [3]:
best_model = load_model('best_deep_model.hdf5')

In [4]:
train_pred = best_model.predict(X_train)
val_pred = best_model.predict(X_val)
test_pred = best_model.predict(X_test)

tn, fp, fn, tp = confusion_matrix(np.round(train_pred),y_train).ravel()
print("Train AUC: {0}, Train Sens: {1}, Train Spec: {2}".format(
    roc_auc_score(y_train,train_pred), tp/(tp+fn), tn/(tn+fp)))
print("Train Acc: {0}".format((tp+tn)/(tp+tn+fp+fn)))

tn, fp, fn, tp = confusion_matrix(np.round(val_pred),y_val).ravel()
print("Val AUC: {0}, Val Sens: {1}, Val Spec: {2}".format(
    roc_auc_score(y_val,val_pred), tp/(tp+fn), tn/(tn+fp)))
print("Val Acc: {0}".format((tp+tn)/(tp+tn+fp+fn)))

tn, fp, fn, tp = confusion_matrix(np.round(test_pred),y_test).ravel()
print("Test AUC: {0}, Test Sens: {1}, Test Spec: {2}".format(
    roc_auc_score(y_test,test_pred), tp/(tp+fn), tn/(tn+fp)))
print("Test Acc: {0}".format((tp+tn)/(tp+tn+fp+fn)))

Train AUC: 0.9882783357697296, Train Sens: 0.9311654861558274, Train Spec: 0.998059248665157
Train Acc: 0.9971995263028534
Val AUC: 0.9861723294069227, Val Sens: 0.9319422776911076, Val Spec: 0.9981718096569692
Val Acc: 0.9973286261541403
Test AUC: 0.98772754285556, Test Sens: 0.9362611737271668, Test Spec: 0.9981264601665313
Test Acc: 0.9973360742224837


In [5]:
nn_test_big = np.concatenate((test_pred,np.reshape(y_test,(y_test.shape[0],1))), axis=1)

In [6]:
np.save("nn_test_big.npy",nn_test_big)